In [1]:
import branca
import clusters_builder
import folium
import json
import importlib
from matplotlib import pyplot as plt
import pandas as pd
import selenium.webdriver

In [2]:
def my_color_function(feature, df_dict, colorscale):
    if not feature['properties']['geoid']  in df_dict:
        return '#ffffff'
    return colorscale(df_dict[feature['properties']['geoid']])

In [3]:
def create_colorscale(n):
    colorscale=branca.colormap.linear.YlOrRd_09.scale(0,n)
    if n>1:
        colorscale=colorscale.to_step(index=[i for i in range(n+1)])
    colorscale.caption='Cluster id'
    return colorscale

In [4]:
def get_geo_json():
    path='data/us-county-boundaries.geojson'
    with open(path) as boundaries:
        geo_json = json.load(boundaries)
        for i in range(len(geo_json['features'])):
            for name in ['Confirmed', 'Deaths']:
                geo_json['features'][i]['properties'][name]=0
        return geo_json

In [5]:
geo_json=get_geo_json()

In [6]:
def modify_geo_json(geo_json, data):
    for i in range(len(geo_json['features'])):
            geoid=int('840'+geo_json['features'][i]['properties']['geoid'])
            for name in ['Confirmed', 'Deaths']:
                if geoid in data['UID'].values:
                        geo_json['features'][i]['properties'][name]=\
                        int(data[data['UID']==geoid][name].values[0])

In [7]:
def create_map(df_dict, n, date):
    colorscale=create_colorscale(n)
    m=folium.Map(location=[36, -97], zoomSnap=0.25, zoom_start=4.75, zoom_control=False)#, zoom_start=2,location=[40., 10.],zoom_control=False, max_bounds=True)
    folium.GeoJson(
        geo_json,
        style_function=lambda feature: {
            'fillColor': my_color_function(feature, df_dict, colorscale),
            'fillOpacity': 1,
            'color' : 'black',
            'weight' : 0.2
            },
        name='COVID CLUSTERS',
        tooltip=folium.features.GeoJsonTooltip(
            fields=['namelsad', 'Confirmed', 'Deaths','stusab'],
            aliases=['County', 'Confirmed', 'Deaths', 'state'])
        ).add_to(m)
    m.add_child(colorscale)
    
    title_html = '''<head><style> html { overflow-y: hidden; } </style></head>''' 
    title_html+='''<h1 align="center"><b>{}</b></h3>'''.format(date)   
    m.get_root().html.add_child(folium.Element(title_html))
    
    m.save('html_us/'+date+'.html')

In [8]:
def save_as_png(date):
    driver = selenium.webdriver.Chrome()
    driver.set_window_size(1800, 800)
    for date in [date]:
        driver.get("file:///C:/Users/DELL/Covid_clusters/html_us/"+date+".html")
        driver.save_screenshot('pictures_us/'+date+'.png')
    driver.quit()

In [9]:
def correct_NY_city(df_dict):
    NY_ID='36061'
    GEO_ID_NY=['36005', '36081', '36047', '36085']
    if NY_ID in df_dict:
        NY_cluster=df_dict[NY_ID]
        for _id in GEO_ID_NY:
            df_dict[_id]=NY_cluster
            

In [10]:
def save_map(builder,date):
    weights, clusters=builder.get_clusters(date)
    plt.hist(weights)
    plt.show()
    print([len(x) for x in clusters])
    print([x for x in clusters if len(x)<10])
    data=builder.loader.data_all_days.loc[builder.loader.data_all_days['Date']==date].copy()
    n_clust=len(clusters)   
    
    df_dict={str(admin_unit[0])[3:]:i+1 for i in range(n_clust) for admin_unit in clusters[i]}
    correct_NY_city(df_dict)
    
    modify_geo_json(geo_json, data)
    #create_map(df_dict, n_clust, date)
    #save_as_png(date)
    

In [19]:
importlib.reload(clusters_builder)
#importlib.reload(inspection)
date='24.03.20'
builder=clusters_builder.Clusters_Builder()

weights, clusters=builder.get_clusters(date)
print([len(x) for x in clusters])
print([x for x in clusters if len(x)<10])
#plt.hist(weights)
#plt.show()
#

438
edges
MST
clust
[434, 1, 1, 1, 1]
[{(84053033, 'King, Washington, US')}, {(84036059, 'Nassau, New York, US')}, {(84036119, 'Westchester, New York, US')}, {(84036061, 'New York City, New York, US')}]


In [12]:
'''
DANGEROUS DO NOT PRESS IF NOT REALLY NEEDED
'''
date='28.06.20'
#save_map(builder, date)

In [13]:
#save_as_png(date)